<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [2]:
import csv

from collections import namedtuple   # Convenient to store the data rows

import pandas as pd

DATA_FILE = './data/chipotle.tsv'
chipotle_file = open(DATA_FILE, 'r')

In [3]:
file_nested_list = csv.reader(chipotle_file, delimiter='\t')

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [4]:
header = None
data = []

In [5]:
for row in file_nested_list:
    if header is None:
        header = row
    else:
        data.append(row)
        
print(header)
#print first 5 rows
print(data[0:5])


['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']
[['1', '1', 'Chips and Fresh Tomato Salsa', 'NULL', '$2.39 '], ['1', '1', 'Izze', '[Clementine]', '$3.39 '], ['1', '1', 'Nantucket Nectar', '[Apple]', '$3.39 '], ['1', '1', 'Chips and Tomatillo-Green Chili Salsa', 'NULL', '$2.39 '], ['2', '2', 'Chicken Bowl', '[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]', '$16.98 ']]


---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [40]:
df = pd.DataFrame(data)
df.columns = header
df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NULL,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [75]:
updated_df = df.copy()
updated_df["item_price"] = updated_df["item_price"].apply(lambda x: x.replace('$',''))
updated_df[["order_id","quantity","item_price"]] = updated_df[["order_id","quantity","item_price"]].apply(pd.to_numeric)
print(updated_df.dtypes)
updated_df.head()

order_id                int64
quantity                int64
item_name              object
choice_description     object
item_price            float64
dtype: object


,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NULL,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98


In [71]:
grouped_df = updated_df.groupby(['order_id','item_name'])['item_price','quantity'].sum()
grouped_df['total_item_price'] = grouped_df['item_price'] * grouped_df['quantity']
grouped_df.groupby('order_id')['total_item_price'].sum().mean()

25.44856597600862

### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [101]:
sodas_bool = (updated_df.loc[:,'item_name'].str.contains(pat = 'Canned Soda')) | (updated_df.loc[:,'item_name'].str.contains(pat = 'Canned Soft Drink'))
sodas = updated_df.loc[sodas_bool,'choice_description'].apply(lambda x: x.replace('[','')).apply(lambda x: x.replace(']','')).unique()
sodas.tolist()

['Sprite',
 'Dr. Pepper',
 'Mountain Dew',
 'Diet Dr. Pepper',
 'Coca Cola',
 'Diet Coke',
 'Coke',
 'Lemonade',
 'Nestea']

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [115]:
burrito_bool = updated_df.loc[:,'item_name'].str.contains(pat = 'Burrito')
burritos = updated_df.loc[burrito_bool,]
burritos.head()

,order_id,quantity,item_name,choice_description,item_price
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",9.25
16,8,1,Chicken Burrito,"[Tomatillo-Green Chili Salsa (Medium), [Pinto ...",8.49
17,9,1,Chicken Burrito,"[Fresh Tomato Salsa (Mild), [Black Beans, Rice...",8.49
21,11,1,Barbacoa Burrito,"[[Fresh Tomato Salsa (Mild), Tomatillo-Green C...",8.99


In [126]:
toppings = burritos['choice_description'].apply(lambda x: x.replace('[','')).apply(lambda x: x.replace(']','')).apply(lambda x: x.split(","))
print(toppings.head())
toppings.apply(lambda x: len(x)).mean()


7     [Tomatillo Red Chili Salsa,  Fajita Vegetables...
9     [Fresh Tomato Salsa,  Rice,  Black Beans,  Pin...
16    [Tomatillo-Green Chili Salsa (Medium),  Pinto ...
17    [Fresh Tomato Salsa (Mild),  Black Beans,  Ric...
21    [Fresh Tomato Salsa (Mild),  Tomatillo-Green C...
Name: choice_description, dtype: object


5.395051194539249

### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [139]:
chips_bool = updated_df.loc[:,'item_name'].str.contains(pat = 'Chip')
chip_orders = updated_df.loc[chips_bool,]
chip_orders = chip_orders.replace({'item_name':{'Side of Chips':'Chips','Chips and Tomatillo-Red Chili Salsa':'Chips and Tomatillo Red Chili Salsa','Chips and Tomatillo-Green Chili Salsa':'Chips and Tomatillo Green Chili Salsa','Chips and Roasted Chili-Corn Salsa':'Chips and Roasted Chili Corn Salsa'}})
chip_orders.groupby('item_name')['quantity'].count().to_dict()

{'Chips': 312,
 'Chips and Fresh Tomato Salsa': 110,
 'Chips and Guacamole': 479,
 'Chips and Mild Fresh Tomato Salsa': 1,
 'Chips and Roasted Chili Corn Salsa': 40,
 'Chips and Tomatillo Green Chili Salsa': 74,
 'Chips and Tomatillo Red Chili Salsa': 68}

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!
